In [4]:
import pandas as pd 
import json 
import os
from tqdm import tqdm
import subprocess
import platform
import threading

from jschon import create_catalog,JSON, JSONSchema
import json,os


In [2]:
df = pd.read_csv('./results/results.csv')
df.head()

,schema,DG_ted,JE_ted,JSF_ted,DG_number_errors,JE_number_errors,JSF_number_errors,correct_instance_ast_size,DG_ast_size,JE_ast_size,JSF_ast_size
0,o1,0.0,14.0,60.0,0.0,0.0,0.0,3,3.0,17.0,73.0
1,o10008,0.0,116.0,1.0,0.0,0.0,0.0,5,5.0,155.0,5.0
2,o10009,0.0,0.0,8.0,0.0,0.0,0.0,1,1.0,1.0,9.0
3,o10010,0.0,0.0,4.0,0.0,0.0,0.0,1,1.0,1.0,5.0
4,o10011,0.0,2.0,10.0,0.0,2.0,0.0,3,3.0,3.0,15.0


In [3]:
df[df['JSF_number_errors']>10000]

,schema,DG_ted,JE_ted,JSF_ted,DG_number_errors,JE_number_errors,JSF_number_errors,correct_instance_ast_size,DG_ast_size,JE_ast_size,JSF_ast_size
2986,o63193,0.0,NaN,154308.0,0.0,NaN,148634.0,1,1.0,NaN,202371.0


In [14]:
CORRECT_INSTANCES_PATH='./dataset/correct-instances'
if not os.path.exists(CORRECT_INSTANCES_PATH): 
    os.mkdir(CORRECT_INSTANCES_PATH)

# 1- Processing data 

since the data is stringified, we need a way to save it into the correct format (json)

In [10]:
df = pd.read_csv("./dataset/01102022_150755_3600000ms_witness.csv")

In [11]:
df.head()

,objectId,witness
0,o27825,true
1,o27841,0
2,o67205,0
3,o89637,0
4,o27830,0


In [81]:
len(df.index)

6335

In [97]:
print(f"Processing : {df.index} correct instances")
for i in tqdm(df.index) :
    objectId,witness = df.loc[i]['objectId'],df.loc[i]['witness']
    if str(witness)=='nan' :
        witness = 'null'   
    # print(objectId,witness)
    json_witness = json.loads(witness)
    with open(f'{CORRECT_INSTANCES_PATH}/{objectId}.json','w',encoding='utf-8') as f : 
        f.write(json.dumps(json_witness))

Processing : RangeIndex(start=0, stop=6335, step=1) correct instances


100%|██████████| 6335/6335 [00:08<00:00, 728.66it/s] 


# 2- Transfering data to bracket notation

In [121]:
BRACKET_DIR = './dataset/bracket'
dirs = ['correct-instances','DG','JE','JSF']
BRACKET_SCRIPT_PATH ='./scripts/prepare.sh'

In [99]:
if not os.path.exists(BRACKET_DIR):
    os.makedirs(BRACKET_DIR)

In [126]:
for dir in dirs : 
    DESTINATION_DIR_BRACKET=f'./dataset/bracket/{dir}'
    if not os.path.exists(DESTINATION_DIR_BRACKET):
        os.makedirs(DESTINATION_DIR_BRACKET)

In [133]:
def process_bracket_directory(dir):
    for file in tqdm(os.listdir(f'./dataset/{dir}') ): 
        file_name = file.split('.')[0]
        
        SOURCE_JSON=f'./dataset/{dir}/{file_name}'
        DESTINATION_DIR_BRACKET=f'./dataset/bracket/{dir}'
        DESTINATION_BRACKET=f'{DESTINATION_DIR_BRACKET}/{file_name}'
        if platform.system() == 'Windows' : 
            output = subprocess.call(['wsl',BRACKET_SCRIPT_PATH,SOURCE_JSON,DESTINATION_BRACKET])
        else : 
            output = subprocess.call([BRACKET_SCRIPT_PATH,SOURCE_JSON,DESTINATION_BRACKET])

In [135]:
threads = []
for dir in dirs:
    thread = threading.Thread(target=process_bracket_directory, args=(dir))
    thread.start()
    threads.append(thread)

# Wait for all threads to finish
for thread in threads:
    thread.join()
print("All directories processed.")

  0%|          | 0/6335 [00:00<?, ?it/s]





  0%|          | 1/6335 [00:01<2:29:33,  1.42s/it]


  0%|          | 2/6335 [00:02<1:55:13,  1.09s/it]

  0%|          | 3/6335 [00:03<1:51:16,  1.05s/it]



  0%|          | 4/6335 [00:05<2:18:32,  1.31s/it]


  0%|          | 5/6335 [00:06<2:07:50,  1.21s/it]


  0%|          | 6/6335 [00:07<2:20:10,  1.33s/it]


  0%|          | 8/6335 [00:09<1:55:18,  1.09s/it]


  0%|          | 9/6335 [00:10<1:49:05,  1.03s/it]


  0%|          | 10/6335 [00:11<1:46:13,  1.01s/it]


  0%|          | 11/6335 [00:12<1:41:22,  1.04it/s]




  0%|          | 12/6335 [00:13<1:46:03,  1.01s/it]


  0%|          | 13/6335 [00:15<2:21:29,  1.34s/it]


  0%|          | 14/6335 [00:16<2:15:54,  1.29s/it]


  0%|          | 15/6335 [00:18<2:38:50,  1.51s/it]


  0%|          | 16/6335 [00:20<2:39:55,  1.52s/it]


  0%|          | 17/6335 [00:20<2:20:53,  1.34s/it]


  0%|          | 18/6335 [00:21<2:11:21,  1.25s/it]


  0%|          | 19/6335 [00:22<1:56:45,  1.11s/it]


  0%|          | 20/6335 [00:24<2

All directories processed.


# 3- Migrating Schema 

In [6]:
MIGRATION_DIR = './dataset/schemas-migrated'
SCHEMAS_DIR ='./dataset/schemas'

In [7]:
create_catalog('2019-09')

Catalog('catalog')

In [8]:
def migrate_schema(schema_migration_path,schema_path):
    try:
        # Convertir le schéma JSON en chaîne JSON
        schema_migration_path_str = json.dumps(schema_migration_path)
        schema_path_str = json.dumps(schema_path)
        # Appeler le script TypeScript avec le schéma JSON en tant qu'argument
        command = ['ts-node', '-e', f'require("./scripts/migrate").migrate({schema_migration_path_str},{schema_path_str})']
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE,shell=True, text=True)
        stdout, stderr = process.communicate()
        # Vérifier si une erreur s'est produite pendant l'exécution du script TypeScript
        if stderr:
            raise RuntimeError(f"Erreur lors de l'exécution du script TypeScript : {stderr}")
        return True

    except Exception as e:
        # print(str(e))
        print(f"Une erreur s'est produite : {e}")
        return None

In [9]:
def migrate(migration_dir:str,schemas_dir:str,schema_name:str): 
    schema_migration_path = f'{migration_dir}/{schema_name}.json'
    schema_path = f'{schemas_dir}/{schema_name}.json'
    try : 
        schema_migrated = migrate_schema(schema_migration_path,schema_path)
        if schema_migrated is None :
            return False 
        return True
    except Exception as e: 
        print(e)
        return False

In [7]:
migrate(MIGRATION_DIR,SCHEMAS_DIR,'o10499')

True

In [10]:
NUM_THREADS = 8

In [13]:
def process_sub_dir(migration_dir,schemas_dir,file_list) : 
    for schema in tqdm(file_list) :
        schema_name = str(schema).split('.')[0]
        print(f"Migrating : {schema_name}") 
        res = migrate(migration_dir,schemas_dir,schema_name)
        if res : 
            print("Migrating correctly")
        else : 
            print("Could not migrate")
            
def migrate_schemas(migration_dir,schemas_dir): 
    if not os.path.exists(migration_dir):
        os.mkdir(migration_dir)
    if not os.path.exists(schemas_dir) or not os.path.isdir(schemas_dir) : 
        raise Exception(f"No directory {schemas_dir}")
    # for schema in tqdm(os.listdir(schemas_dir)) :
    
    files = os.listdir(schemas_dir)
    chunk_size = len(files) // NUM_THREADS
    
    # Créer et démarrer les threads
    threads = []
    for i in range(NUM_THREADS):
        lower_bound = i * chunk_size
        upper_bound = (i + 1) * chunk_size if i < NUM_THREADS - 1 else len(files)
        thread = threading.Thread(target=process_sub_dir, args=(migration_dir, schemas_dir, files[lower_bound:upper_bound]))
        thread.start()
        threads.append(thread)
    
    # Attendre que tous les threads se terminent
    for thread in threads:
        thread.join()
    

In [14]:
migrate_schemas(MIGRATION_DIR,SCHEMAS_DIR)

  0%|          | 0/797 [00:00<?, ?it/s]

Migrating : o1
Migrating : o3461


Migrating : o21306


Migrating : o44284


Migrating : o57432


...

Migrating correctly
Migrating : o74593


Migrating correctly
Migrating : o74594


Migrating correctly
Migrating : o74595







100%|██████████| 797/797 [3:31:33<00:00, 15.93s/it]

Migrating correctly


In [20]:
len(os.listdir(SCHEMAS_DIR)), len(os.listdir(MIGRATION_DIR))

(6377, 6374)